In [1]:
import numpy as np

In [2]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

In [3]:
from xgboost import XGBClassifier
# xgb = XGBClassifier(n_estimators=20, random_state=71)
from sklearn.linear_model import LogisticRegression
# lr = LogisticRegression(solver='lbfgs', max_iter=300)
from sklearn import neighbors
# knn = neighbors.KNeighborsClassifier(5, weights = 'distance')
from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier()

In [4]:
# NN
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Activation
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.
/Users/kumagaimayuko/opt/anaconda3/envs/myAnalysis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/kumagaimayuko/opt/anaconda3/envs/myAnalysis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/kumagaimayuko/opt/anaconda3/envs/myAnalysis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype

In [5]:
from data_cleaning import CleanTW
tw = CleanTW().run_all()

In [6]:
print(tw.columns)
print(tw.shape)
tw.head()

Index(['ツイート本文', 'ツイート本文_x', '時間', 'いいね', 'リツイート', 'インプレッション', 'エンゲージメント',
       'エンゲージメント率', '返信', 'ユーザープロフィールクリック', 'URLクリック数', 'ハッシュタグクリック', '詳細クリック',
       'フォローしている', 'メディアの再生数', 'メディアのエンゲージメント数', '文字数', 'media_exist',
       'reply_flg', 'YMD', 'YEAR', 'MONTH', 'DAY', 'TIME', 'WEEKDAY', 'WEEK'],
      dtype='object')
(945, 26)


ツイート本文  \
0                     @teatime3fairy こちらこそ宜しくお願いします😆   
1                    ねむねむむむ🥱 https://t.co/Ggro68dOXM   
2  こっちに来るかと思いきや\nそれちゃって💦\nパニック映像になった😂😂 https://t....   
3                    @santemafi ありがとうございます😊\n頑張りました😆   
4            @hitomin_rabi 頑張りました😊\nこの後いっぱいおやつあげました🍭   

                             ツイート本文_x                        時間    いいね  リツイート  \
0                     こちらこそ宜しくお願いします😆 2021-03-30 14:34:00+00:00    1.0    0.0   
1                            ねむねむむむ🥱  2021-03-30 07:57:00+00:00  216.0    3.0   
2  こっちに来るかと思いきや それちゃって💦 パニック映像になった😂😂  2021-03-25 12:07:00+00:00  107.0    0.0   
3                 ありがとうございます😊 頑張りました😆 2021-03-25 02:53:00+00:00    1.0    0.0   
4            頑張りました😊 この後いっぱいおやつあげました🍭 2021-03-25 02:53:00+00:00    1.0    0.0   

   インプレッション  エンゲージメント  エンゲージメント率   返信  ユーザープロフィールクリック  ...  文字数  media_exist  \
0      14.0       1.0   0.071429  0.0             0.0  ...   16            0   
1    2654.0     331.0   0.124717  0.0            44.0  ...    8            1   
2    1362.0     199.0   0.146109  0.0            17.0  ...   34            1   
3      35.0       1.0   0.028571  0.0             0.0  ...   20            0   
4      37.0       2.0   0.054054  1.0             0.0  ...   25            0   

   reply_flg       YMD  YEAR  MONTH  DAY TIME WEEKDAY WEEK  
0          1  20210330  2021      3   30   14       1   13  
1          0  20210330  2021      3   30    7       1   13  
2          0  20210325  2021      3   25   12       3   12  
3          1  20210325  2021      3   25    2       3   12  
4          1  20210325  2021      3   25    2       3   12  

[5 rows x 26 columns]

In [9]:
train_x = tw[['リツイート', 'インプレッション', 'エンゲージメント',
       'エンゲージメント率', '返信', 'ユーザープロフィールクリック', 'URLクリック数', 'ハッシュタグクリック', '詳細クリック',
       'フォローしている', 'メディアの再生数', 'メディアのエンゲージメント数', '文字数']].copy()
train_y = tw['いいね'].copy()

## xgboost（勾配ブースティング）

In [10]:
#xgb

# 各foldのスコアを保存するリスト
scores_accuracy = []

# クロスバリデーションを行う
# 学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    # 学習データを学習データとバリデーションデータに分ける
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # モデルの学習を行う
    xgb = XGBClassifier(n_estimators=20, random_state=71)
    xgb.fit(tr_x, tr_y)

    # バリデーションデータの予測値を確率で出力する
    va_pred = xgb.predict_proba(va_x)[:, 1]

    # バリデーションデータでのスコアを計算する
    accuracy = accuracy_score(va_y, va_pred > 0.5)

    # そのfoldのスコアを保存する
    scores_accuracy.append(accuracy)

# 各foldのスコアの平均を出力する
accuracy = np.mean(scores_accuracy)
print(f'accuracy: {accuracy:.4f}')

accuracy: 0.4984


In [11]:
#lr

# 各foldのスコアを保存するリスト
scores_accuracy = []

# クロスバリデーションを行う
# 学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    # 学習データを学習データとバリデーションデータに分ける
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # モデルの学習を行う
    lr = LogisticRegression(solver='lbfgs', max_iter=300)
    lr.fit(train_x, train_y)
    va_pred = lr.predict_proba(va_x)[:, 1]

    # バリデーションデータでのスコアを計算する
    accuracy = accuracy_score(va_y, va_pred > 0.5)

    # そのfoldのスコアを保存する
    scores_accuracy.append(accuracy)

# 各foldのスコアの平均を出力する
accuracy = np.mean(scores_accuracy)
print(f'accuracy: {accuracy:.4f}')

accuracy: 0.2465


In [12]:
#knn

# 各foldのスコアを保存するリスト
scores_accuracy = []

# クロスバリデーションを行う
# 学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    # 学習データを学習データとバリデーションデータに分ける
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # モデルの学習を行う
    knn = neighbors.KNeighborsClassifier(5, weights = 'distance')
    knn.fit(tr_x, tr_y)

    # バリデーションデータの予測値を確率で出力する
    va_pred = knn.predict(va_x)

    # バリデーションデータでのスコアを計算する
    accuracy = accuracy_score(va_y, va_pred > 0.5)

    # そのfoldのスコアを保存する
    scores_accuracy.append(accuracy)

# 各foldのスコアの平均を出力する
accuracy = np.mean(scores_accuracy)
print(f'accuracy: {accuracy:.4f}')

accuracy: 0.4085


In [13]:
#rf

# 各foldのスコアを保存するリスト
scores_accuracy = []

# クロスバリデーションを行う
# 学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    # 学習データを学習データとバリデーションデータに分ける
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # モデルの学習を行う
    rf = RandomForestClassifier()
    rf.fit(tr_x, tr_y)

    # バリデーションデータの予測値を確率で出力する
    va_pred = rf.predict_proba(va_x)[:, 1]

    # バリデーションデータでのスコアを計算する
    accuracy = accuracy_score(va_y, va_pred > 0.5)

    # そのfoldのスコアを保存する
    scores_accuracy.append(accuracy)

# 各foldのスコアの平均を出力する
accuracy = np.mean(scores_accuracy)
print(f'accuracy: {accuracy:.4f}')

accuracy: 0.4984


In [14]:
import matplotlib.pyplot as plt

fti = rf.feature_importances_

print('Feature Importances:')
for i, feat in enumerate(train_x.columns):
    print('\t{0:20s} : {1:>.6f}'.format(feat, fti[i]))

Feature Importances:
	リツイート                : 0.042324
	インプレッション             : 0.129834
	エンゲージメント             : 0.198241
	エンゲージメント率            : 0.170256
	返信                   : 0.041528
	ユーザープロフィールクリック       : 0.102807
	URLクリック数             : 0.034939
	ハッシュタグクリック           : 0.018223
	詳細クリック               : 0.071423
	フォローしている             : 0.002609
	メディアの再生数             : 0.056191
	メディアのエンゲージメント数       : 0.059837
	文字数                  : 0.071788


## ARIMAで時系列分析

## NN・RNN

In [15]:
#nn

# 各foldのスコアを保存するリスト
scores_accuracy = []

# クロスバリデーションを行う
# 学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    # 学習データを学習データとバリデーションデータに分ける
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # モデルの学習を行う
    nn = Sequential()
    nn.add(Dense(30, input_dim=train_x.shape[1]))
    nn.add(Activation('relu'))
    nn.add(Dense(10))
    nn.add(Activation('softmax'))

    nn.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    Y_train = to_categorical(train_y, 10)

    hist = nn.fit(train_x, Y_train,
                     epochs=30, batch_size=100,verbose=0)

    # バリデーションデータの予測値を確率で出力する
    va_pred = nn.predict_classes(va_x, batch_size=1)

    # バリデーションデータでのスコアを計算する
    accuracy = accuracy_score(va_y, va_pred > 0.5)

    # そのfoldのスコアを保存する
    scores_accuracy.append(accuracy)

# 各foldのスコアの平均を出力する
accuracy = np.mean(scores_accuracy)
print(f'accuracy: {accuracy:.4f}')

IndexError: index 216 is out of bounds for axis 1 with size 10

## ベイズ 混合効果モデル

In [29]:
# 曜日によって文字数がいいね数に与える影響が異なるとして、交互作用を用いた混合効果モデルを作成
# formula = いいね ~ WEEKDAY * 文字数
tw.groupby(['WEEKDAY','文字数'])['いいね'].agg(['size','mean','median']).head(30)

size        mean  median
WEEKDAY 文字数                          
0       0       1  375.000000   375.0
        6       2    0.500000     0.5
        7       3  473.666667   462.0
        8       5   68.000000     1.0
        9       2  256.000000   256.0
        10      5  227.000000     1.0
        11      3   72.000000     1.0
        12     23   17.782609     1.0
        13      4   86.750000     1.0
        14      3  289.666667   240.0
        15      3  243.333333   273.0
        16      5   82.200000     1.0
        17      1    1.000000     1.0
        18      3  280.000000   403.0
        19     15   13.133333     1.0
        21      1    1.000000     1.0
        22      1    1.000000     1.0
        23      1  279.000000   279.0
        24      1  351.000000   351.0
        25      2  250.500000   250.5
        26      2   64.000000    64.0
        27      5    3.600000     1.0
        28      2  125.500000   125.5
        29      3    1.000000     1.0
        30      4  297.500000     1.0
        31      3   68.333333     1.0
        32      1    0.000000     0.0
        33      2    1.000000     1.0
        35      5  196.800000     1.0
        36      1    1.000000     1.0

In [28]:
# 時間帯によって文字数がいいね数に与える影響が異なるとして、交互作用を用いた混合効果モデルを作成
# formula = いいね ~ Time * 文字数
tw.groupby(['TIME','文字数'])['いいね'].agg(['size','mean','median']).head(30)

size        mean  median
TIME 文字数                          
0    0       1  305.000000   305.0
     3       1    1.000000     1.0
     6       1    1.000000     1.0
     7       1    0.000000     0.0
     8       1  187.000000   187.0
     9       3    3.666667     2.0
     10      1    0.000000     0.0
     11      2    1.000000     1.0
     12      6    0.666667     1.0
     13      2    0.000000     0.0
     14      1    0.000000     0.0
     16      1    0.000000     0.0
     17      1    1.000000     1.0
     21      2  343.000000   343.0
     22      1    1.000000     1.0
     23      1    0.000000     0.0
     28      2  171.000000   171.0
     29      1    0.000000     0.0
     30      2  264.500000   264.5
     33      2  428.000000   428.0
     35      1  322.000000   322.0
     36      1  234.000000   234.0
     37      1  234.000000   234.0
     55      1  472.000000   472.0
     62      1  125.000000   125.0
     66      1    1.000000     1.0
     78      1   30.000000    30.0
     139     1  867.000000   867.0
1    6       1  397.000000   397.0
     7       1    1.000000     1.0